![Egeria Logo](https://raw.githubusercontent.com/odpi/egeria/master/assets/img/ODPi_Egeria_Logo_color.png)

### Egeria Hands-On Lab
# Welcome to the UI Asset Search Lab

## Introduction

Egeria is an open source project that provides open standards and implementation libraries to connect tools, catalogs and platforms together so they can share information (called metadata) about data and the technology that supports it.

In this hands-on lab you will get a chance to have hands-on experience with the UI of Egeria, by searching for assets we add along the way.

## The Scenario

The ODPi Egeria team use the personas and scenarios from the fictitious company called Coco Pharmaceuticals.  (See https://opengovernance.odpi.org/coco-pharmaceuticals/ for more information).

As part of the huge business transformation that Coco Pharmaceuticals has embarked on, they
have created a data lake for managing data for research, analytics, exchange between their internal organizations and business partners (such as hospitals).  As a result, the data lake has to be
designed to handle a wide variety of data, including some highly sensitive and regulated data.

In this lab we first create some assets. Afterwards, the UI where we can search for assets is shown. The main characters engaged in this lab are the data analyst named [Peter Profile](https://github.com/odpi/data-governance/blob/master/docs/coco-pharmaceuticals/personas/peter-profile.md), and the data scientist named [Callie Quartile](https://github.com/odpi/data-governance/blob/master/docs/coco-pharmaceuticals/personas/callie-quartile.md).

<figure style="margin-left: 7%; display:inline-block;">  
  <img src="../images/peter-profile.png">
  <figcaption style="margin-left: 15%;"><strong>Peter Profile</strong></figcaption>
</figure>

<figure style="margin-left: 20%; display:inline-block;">  
  <img src="../images/callie-quartile.png">
  <figcaption style="margin-left: 15%;"><strong>  Callie Quartile</strong></figcaption>
</figure>



## Setting up

Coco Pharmaceuticals make widespread use of ODPi Egeria for tracking and managing their data and related assets.
Figure 1 below shows their metadata servers and the Open Metadata and Governance (OMAG) Server Platforms that are hosting them.  Each metadata server supports a department in the organization.  The servers are distributed across the platform to even out the workload.  Servers can be moved to a different platform if needed.

![Figure 1](../images/coco-pharmaceuticals-systems-omag-server-platforms.png)
> **Figure 1:** Coco Pharmaceuticals' OMAG Server Platforms

The code below sets up the network addresses for the three platforms.  This varies depending on whether you are running them locally, through **docker-compose** or on **kubernetes**.

In [ ]:
%run ../common/environment-check.ipynb

----
Callie is using the data lake metadata server called `cocoMDS4`. This server is hosted on the Data Lake OMAG Server Platform. It enables business users and the executive team to access data from the data lake.

Check that `cocoMDS4` is running.  

If any of the platforms are not running, follow [this link to set up and run the platform](https://egeria.odpi.org/open-metadata-resources/open-metadata-labs/).  If any server is reporting that it is not configured then
run the steps in the **[Server Configuration](../egeria-server-config.ipynb)** lab to configure
the servers.  Then re-run the previous step to ensure all of the servers are started.

----
## Exercise 1

### Generating some assets

Before we can interact with the UI, we have to create some data in the metadata repository. This section is a simplified version of the [Building a Data Catalog Lab](../asset-management-labs/building-a-data-catalog.ipynb). If you haven't done so already, it is recommended that you also check those labs out.

For this part of the exercises, Peter is going to create some CSV files using Asset Owner OMAS located in `cocoMDS1`. The code block below describes their basic attributes.

In [ ]:
csv_file_list = [
    {
        "displayName": "List of patients",
        "description": "Basic information regarding patients recorded in February 2020.",
        "fullPath": "file://secured/research/patients.csv"
    },
    {
        "displayName": "Log of treatments",
        "description": "Treatments carried out for patients in 2019.",
        "fullPath": "file://secured/research/treatments.csv"
    },
]

Afterwards, we will have our Data Analyst, [Peter Profile](https://github.com/odpi/data-governance/blob/master/docs/coco-pharmaceuticals/personas/peter-profile.md), add these CSV files into the repository. You can see two lists of GUIDs for the two CSV files we have added. For a more thorough explanation for this step, check out the [Building a Data Catalog lab](../asset-management-labs/building-a-data-catalog.ipynb).

In [ ]:
for csv_file_specs in csv_file_list:
    response_guid = assetOwnerCreateCSVAsset(
        serverName=cocoMDS1Name, 
        serverPlatformName=cocoMDS1PlatformName, 
        serverPlatformURL=cocoMDS1PlatformURL, 
        userId=petersUserId,
        **csv_file_specs
    )
    printGUIDList(response_guid)

## Exercise 2

### Search for assets in the UI

Now we can get the URL through the environment variable, with a default value of `https://localhost:8443`:

In [ ]:
uiURL = os.environ.get('uiExternalURL', 'https://localhost:8443')
print(uiURL)

Viting the URL leads us to the Egeria UI

<img src="../images/egeria-ui-login.png" width="800" height="100" alt="Figure 2" />

> **Figure 2:** Egeria UI's login page

and we can login with the credential of Callie Quartile:
```
Username: calliequartile
Password: admin
```

---
After logging in, we should end up at the asset search page:

<img src="../images/egeria-ui-asset-search.png" width="800" height="100" alt="Figure 3" />

> **Figure 3:** Egeria UI's asset search page

---

Next, searching for `CSV` yields the results as follows:

<img src="../images/egeria-ui-asset-list.png" width="800" height="100" alt="Figure 4" />

> **Figure 4:** Egeria UI's asset search result

The following attributes are present:
- **Name**: Name of the asset.
- **Type**: Asset type.
- **Description**: Text description of the asset. In our example this field is empty for all four results.
- **QualifiedName**: Unique identifier to differentiate different assets.
- **Options**: Additional interactive options to interact with the returned assets. The `Lineage` hyperlink is clickable, but asset lineage is not within the scope of this tutorial.